In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive as mountGoogleDrive 
mountGoogleDrive.mount('/content/drive')

Mounted at /content/drive


In [18]:
out_dir = r'/content/drive/MyDrive/Research/Safegraph/scratch/20221205/'

# Get Data

In [3]:
uhf_map = pd.read_csv('https://raw.githubusercontent.com/nychealth/coronavirus-data/master/Geography-resources/UHF_resources/uhf-to-zip%20crosswalk.csv')

In [4]:
uhf_map

,UHF_id,UHF_name,Zipcodes
0,101,Kingsbridge - Riverdale,"10463,10471"
1,102,Northeast Bronx,"10466,10469,10475"
2,103,Fordham - Bronx Pk,"10458,10467,10468,10470"
3,104,Pelham - Throgs Neck,"10461,10462,10464,10465,10472,10473"
4,105,Crotona -Tremont,"10453,10457,10460"
5,106,High Bridge - Morrisania,"10451,10452,10456"
6,107,Hunts Point - Mott Haven,"10454,10455,10459,10474"
7,201,Greenpoint,"11211,11222"
8,202,Downtown - Heights - Slope,"11201,11205,11215,11217,11231"
9,203,Bedford Stuyvesant - Crown Heights,"11212,11213,11216,11233,11238"


In [5]:
modzcta_map = {}

for uhf_id in np.unique(uhf_map['UHF_id']):
  modzcta_ids = uhf_map.loc[uhf_map['UHF_id'] == uhf_id,'Zipcodes'].values[0].split(",")
  for m in modzcta_ids:
    modzcta_map[int(m)] = uhf_id

modzcta_map = pd.DataFrame.from_dict(modzcta_map, orient='index', columns=['UHF_id']).reset_index()
modzcta_map = modzcta_map.rename(columns={"index": "MODZCTA"})

In [6]:
cases_df = pd.read_csv('https://raw.githubusercontent.com/nychealth/coronavirus-data/master/trends/caserate-by-modzcta.csv')
drop_cols = ['CASERATE_CITY', 'CASERATE_BX', 'CASERATE_BK', 'CASERATE_MN', 'CASERATE_QN', 'CASERATE_SI']
cases_df.drop(drop_cols, axis=1, inplace=True)
cases_df = pd.wide_to_long(cases_df, stubnames='CASERATE_', i=['week_ending'], j='MODZCTA')
cases_df = cases_df.reset_index()

In [7]:
pop_df = pd.read_csv(r'/content/drive/MyDrive/Research/Safegraph/scratch/Modified_Zip_Code_Tabulation_Areas__MODZCTA_.csv')

In [8]:
cases_df = cases_df.merge(pop_df[['MODZCTA', 'pop_est']], on='MODZCTA', how='left')

In [9]:
cases_df['case_count'] = cases_df['CASERATE_'] * cases_df['pop_est'] / 100_000

In [10]:
cases_df = cases_df.merge(modzcta_map, on='MODZCTA', how='left')

In [11]:
cases_df = cases_df.groupby(['week_ending', 'UHF_id'])['case_count'].sum().reset_index()

In [12]:
cases_df['week_ending'] = pd.to_datetime(cases_df.week_ending, format='%m/%d/%Y')

In [21]:
cases_df.to_csv(f"{out_dir}/cases.csv", index=False)

In [14]:
pop_df = pop_df.merge(modzcta_map, on='MODZCTA', how='left')

In [15]:
uhf_pop_df = pop_df.groupby('UHF_id')['pop_est'].sum()

In [20]:
uhf_pop_df.reset_index().to_csv(f"{out_dir}/pop.csv", index=False)